In [124]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Zipline, PyAlgoTrader, Catalyst, Backtrader, Vectorbt

# Simple Moving Average function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

# SMA Cross Strategy class
class SmaCross(Strategy):
    n1 = 6
    n2 = 8

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



In [125]:
import yfinance as yf

# Get BTC-USD data from yfinance
#data = yf.download('BTC-USD')
data = yf.download(tickers='USD',start='2024-04-01', end='2024-04-30', interval='1d')

# Code for running the backtest.
# Assumes the existence of `data` variable containing backtest data.
bt = Backtest(data, SmaCross)
stats = bt.run()


[*********************100%%**********************]  1 of 1 completed


In [126]:
print(stats['_trades'].to_string())
print(stats)
#stats


   Size  EntryBar  ExitBar  EntryPrice  ExitPrice         PnL  ReturnPct  EntryTime   ExitTime Duration
0   110        11       12   90.379997  92.809998  267.300034   0.026886 2024-04-16 2024-04-17   1 days
1  -110        12       20   92.809998  90.360001  269.499664   0.026398 2024-04-17 2024-04-29  12 days
Start                     2024-04-01 00:00:00
End                       2024-04-29 00:00:00
Duration                     28 days 00:00:00
Exposure Time [%]                   47.619048
Equity Final [$]                 10628.439804
Equity Peak [$]                  12411.199799
Return [%]                           6.284398
Buy & Hold Return [%]               -9.267368
Return (Ann.) [%]                  107.794576
Volatility (Ann.) [%]              140.866083
Sharpe Ratio                         0.765227
Sortino Ratio                         3.31582
Calmar Ratio                         7.010009
Max. Drawdown [%]                  -15.377239
Avg. Drawdown [%]                  -15.37723

In [127]:
bt.plot()

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p7041', ...)

In [128]:
# Define a range of values to test for each parameter
param_grid = {'n1': range(5, 30, 1), 'n2': range(5, 30, 1)}
# Run the optimization
res = bt.optimize(**param_grid)

# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 625 configurations.
  output = _optimize_grid()
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by

Best result:  6.284398040771484
Parameters for best result:  SmaCross(n1=6,n2=8)


In [129]:
res

Start                     2024-04-01 00:00:00
End                       2024-04-29 00:00:00
Duration                     28 days 00:00:00
Exposure Time [%]                   47.619048
Equity Final [$]                 10628.439804
Equity Peak [$]                  12411.199799
Return [%]                           6.284398
Buy & Hold Return [%]               -9.267368
Return (Ann.) [%]                  107.794576
Volatility (Ann.) [%]              140.866083
Sharpe Ratio                         0.765227
Sortino Ratio                         3.31582
Calmar Ratio                         7.010009
Max. Drawdown [%]                  -15.377239
Avg. Drawdown [%]                  -15.377239
Max. Drawdown Duration       10 days 00:00:00
Avg. Drawdown Duration       10 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       2.688648
Worst Trade [%]                      2.639799
Avg. Trade [%]                    

In [130]:
res['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2024-04-01,10000.000000,0.000000,NaT
2024-04-02,10000.000000,0.000000,NaT
2024-04-03,10000.000000,0.000000,NaT
2024-04-04,10000.000000,0.000000,NaT
2024-04-05,10000.000000,0.000000,NaT
2024-04-08,10000.000000,0.000000,NaT
2024-04-09,10000.000000,0.000000,NaT
2024-04-10,10000.000000,0.000000,NaT
2024-04-11,10000.000000,0.000000,NaT
2024-04-12,10000.000000,0.000000,NaT
